In [2]:
import pandas as pd
import numpy as np
import re,glob,os,secrets
import urllib.parse
import urllib.request
from multiprocessing.pool import ThreadPool
import urllib
import pandas as pd
import operator
import functools
from operator import methodcaller

URL = 'https://www.uniprot.org/uploadlists/'

def create_chunks(list_name, n):
    for i in range(0, len(list_name), n):
        yield list_name[i:i + n]
        
def mapping_ids(ids2map, source_fmt='P_GI',target_fmt='ACC', output_fmt='tab'):
    if hasattr(ids2map, 'pop'):
            ids2map = ' '.join(ids2map)
    params = {
    'from': source_fmt,
    'to': target_fmt,
    'format': output_fmt,
    'query': ids2map
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(URL, data)
    with urllib.request.urlopen(req) as f:
        response = f.read().decode('utf-8')
    return response

def concat_dfs(path,extension):
    extension = extension
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    combined_dfs = pd.concat([pd.read_table(f) for f in all_filenames ])
    
    return combined_dfs

def taxonomy_metag(basta_output, besthits):
    columns_basta_output = ['id_read','taxonomy']
    basta_output = pd.DataFrame(np.vstack([basta_output.columns,basta_output]))
    basta_output.columns = columns_basta_output
    taxonomy = basta_output["taxonomy"].str.split(";",expand=True)
    basta_output["taxonomy"] = taxonomy[5]
    besthits = pd.merge(besthits,basta_output,left_on="query_name",right_on="id_read",how="left")
    besthits = besthits.drop("id_read",1)
    besthits = besthits.loc[besthits["taxonomy"]!="unknown"]
    besthits = besthits.loc[besthits["taxonomy"].notna()]
    besthits = besthits.reset_index(drop=True)
    besthits = besthits[~besthits.duplicated(['KO','taxonomy'])].reset_index(drop=True)
    return besthits

In [22]:
df_besthits = pd.read_table("C:/Users/eulle/Documents/metagenomica/marbella/besthits_E7.m8")

columns_csv = ['qseqid' ,'sseqid' ,'pident' ,'length','mismatch','gapopen','qstart' ,'qend' ,'sstart' ,'send' ,'evalue' ,'bitscore']
row_values = pd.Series(df_besthits.columns)
df_besthits = pd.DataFrame(np.vstack([df_besthits.columns,df_besthits]))
df_besthits.columns = columns_csv


In [23]:
gi_number = [re.findall(r"(?<=GI:|gi:).*?(?=\||p)", ii[1]['sseqid']) for ii in df_besthits.iterrows()]
gi_number = functools.reduce(operator.iconcat, gi_number, [])
gi_number = list(set(gi_number))

uniprot_id = [re.findall(r"(?<=BL:|ot:).*?(?=\||p)",ii[1]['sseqid']) for ii in df_besthits.iterrows()]
uniprot_id = functools.reduce(operator.iconcat, uniprot_id, [])
uniprot_id = list(set(uniprot_id))

chunk_gi = list(create_chunks(gi_number,1000))
chunk_uniprot = list(create_chunks(uniprot_id,1000))

In [24]:
# Criar uma função para escrever os arquivos
map_gi = ThreadPool(10).imap(mapping_ids,chunk_gi)
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

parent_dir = "D:/eulle/importante/"
dir_gi_uni = "gi_number-uni"
dir_uni_ko = "uniprot-ko_id"

if(os.path.isdir('D:/eulle/importante/gi_number-uni/')==False):
    os.mkdir(os.path.join(parent_dir, dir_gi_uni))
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/gi_number-uni/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
else:
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/gi_number-uni/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
            
            
if(os.path.isdir('D:/eulle/importante/uniprot-ko_id/')==False):        
    os.mkdir(os.path.join(parent_dir, dir_uni_ko))
    for index,ii in enumerate(map_uniprot): 
        with open('D:/eulle/importante/uniprot-ko_id/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
else:
    for index,ii in enumerate(map_gi):
        with open('D:/eulle/importante/uniprot-ko_id/' + secrets.token_hex(15) +'.csv', 'w') as file:
            file.write(ii)
    

In [25]:
# Pensar em uma forma de melhorar esses downloads
gi_uniprot_concat = concat_dfs(os.chdir("D:/eulle/importante/gi_number-uni/"),"csv")
gi_uniprot_concat = gi_uniprot_concat.drop_duplicates("From")
gi_uniprot_concat = gi_uniprot_concat.reset_index(drop=True)
gi_uniprot_concat.columns = ["GI","Uniprot_ID"]

gi_uniprot_concat.to_csv("D:/eulle/importante/df_gi_uniprot-E7.csv",sep="\t",encoding="utf-8",index=False)

list_gi_uniprot = list(gi_uniprot_concat["Uniprot_ID"])
chunk_uniprot = list(create_chunks(list_gi_uniprot,1000))
map_uniprot = ThreadPool(10).imap(functools.partial(mapping_ids,source_fmt='ACC',target_fmt='KO_ID'),chunk_uniprot)

for index,ii in enumerate(map_uniprot): 
    with open('D:/eulle/importante/uniprot-ko_id/'+ secrets.token_hex(15) +'.csv', 'w') as file:
        file.write(ii)



In [26]:
uniprot_ko_concat = concat_dfs(os.chdir("D:/eulle/importante/uniprot-ko_id/"),"csv")
uniprot_ko_concat.columns = ["Uniprot_ID","KO"]
uniprot_ko_concat = uniprot_ko_concat.reset_index(drop=True)

uniprot_ko_concat.to_csv("D:/eulle/importante/df_ko_ids_E7.csv",sep="\t",encoding="utf-8",index=False)

In [2]:
basta_output = pd.read_fwf("C:/Users/eulle/Documents/metagenomica/besthit_CoSQG.txt")
besthits = pd.read_csv("C:/Users/eulle/Documents/metagenomica/besthits_pathways-CoSQG-julia.csv",sep="]")
taxmy = taxonomy_metag(basta_output,besthits)
#taxmy.to_csv("C:/Users/eulle/Documents/metagenomica/taxonomy_besthits-CoSQG.csv",sep="]",encoding="utf-8",index=False)


# É SÓ PEGAR O UNIQUE DA TABELA TAXONOMY E EXCLUIR OS UNKNOWN -- FEITO
# SABER QUAIS KOs NÀO FORAM MAPEADOS E CONTAR

In [1]:
taxmy.to_csv("C:/Users/eulle/Documents/metagenomica/taxonomy_besthits-CoSQG.csv",sep="]",encoding="utf-8",index=False)

NameError: name 'taxmy' is not defined

In [36]:
taxmy = pd.read_csv("D:/Downloads/df_eggnog_final.csv",sep=",")
taxmy = taxmy[~taxmy.duplicated(['KEGG_ko','best_tax_level'])].reset_index(drop=True)
#taxmy.drop_duplicates('KEGG_ko',inplace=True)

#taxmy.to_csv("D:/Downloads/taxmy_CoSqG.csv",encoding="utf-8",sep=",",index=False)

In [41]:
taxmy = taxmy[pd.isna(taxmy["KEGG_Pathway"])==False].reset_index(drop=True)


In [42]:
taxmy.to_csv("D:/Downloads/taxmy_CoSqG",encoding="utf-8",sep="\t",index=False)